In [1]:
import tensorflow as tf

#### 1. 保存计算两个变量和的模型。

In [2]:
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "v2")
result = v1 + v2

init_op = tf.global_variables_initializer()
# 声明tf.train.Saver类用于保存模型
# 可以指定saver保存什么变量，如果只保存v1，那么下边要恢复result就会出错，因为缺少v2
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, "Saved_model/model.ckpt")
    writer = tf.summary.FileWriter('Saved_Graph',graph=tf.get_default_graph())
    writer.close()

上述代码在Saved_model文件夹下生成4个文件：

```
▶ tree Saved_model
Saved_model
├── checkpoint
├── model.ckpt.data-00000-of-00001
├── model.ckpt.index
└── model.ckpt.meta
```

* meta文件：包含tf计算图的结构，包含变量、op、集合等信息
* cpkt文件：二进制文件，保存了网络中所有权重、偏置等变量数值，分为两个文件:
  * 一个是.data-00000-of-00001 文件
  * 一个是.index 文件
* checkpoint文件：文本文件，记录了最新保持的5个模型文件列表

In [ ]:
!tensorboard --logdir=Saved_Graph

W0905 10:47:11.151640 Reloader tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0905 10:47:11.151640 123145422454784 tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0905 10:47:11.152345 Reloader tf_logging.py:120] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0905 10:47:11.152345 123145422454784 tf_logging.py:120] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0905 10:47:11.162157 Reloader tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0905 10:47:11.162157 1231454

在浏览器上输入网址：127.0.0.1:6006 即可进入tensorboard

图片展示如下：
![image](image/4.1_Saved_Graph.png)

#### 2. 加载保存了两个变量和的模型。
代码和上面保存模型基本一样，先定义计算图上的运算，并声明了一个tf.train.Saver类。

不同点在于，加载模型的代码中没有运行变量的初始化过程，而是将变量的值通过保存的模型加载出来。

默认保存和加载了tf计算图上定义的全部变量。

**加载部分变量**

保存和加载部分变量，在声明tf.train.Saver类时可以提供一个列表来指定需要保存或加载的变量。
```
tf.train.Saver([v1])
```
这里只有v1会被加载进来，

```
FailedPreconditionError: Attempting to use uninitialized value v2
```

**注意**：在 saver.restore()加载的前面，需要添加 `tf.reset_default_graph()`，作用是 清除默认图的堆栈，并设置全局图为默认图

In [7]:
# 清除默认图的堆栈，并设置全局图为默认图 
tf.reset_default_graph()

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "v2")
result = v1 + v2

saver = tf.train.Saver()
#saver = tf.train.Saver([v1])

with tf.Session() as sess:
    saver.restore(sess, "Saved_model/model.ckpt")
    print(result.eval())

INFO:tensorflow:Restoring parameters from Saved_model/model.ckpt
[3.]


#### 3. 如果不希望重复定义图上的运算，可以直接加载持久化的图：tf.train.import_meta_graph。

In [8]:
# 清除默认图的堆栈，并设置全局图为默认图 
tf.reset_default_graph()

saver = tf.train.import_meta_graph("Saved_model/model.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess, "Saved_model/model.ckpt")
    print(sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))

INFO:tensorflow:Restoring parameters from Saved_model/model.ckpt
[3.]


#### 4. 变量重命名。
除了可以选取需要被加载的变量，tf.train.Saver类也支持在保存或加载时给变量重命名。

In [13]:
# 声明的变量名和已保存模型中变量名不同
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "other-v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "other-v2")
result = v1 + v2

# 如果直接加载，会出现变量找不到的错误
# NotFoundError: Key other-v1 not found in checkpoint
# saver = tf.train.Saver()

# 对变量重命名后可以找到
saver = tf.train.Saver({"v1": v1, "v2": v2})

with tf.Session() as sess:
    saver.restore(sess, "Saved_model/model.ckpt")
    print(result.eval())

INFO:tensorflow:Restoring parameters from Saved_model/model.ckpt
[3.]
